In [1]:
import collections
import math
import torch
from torch import nn
from d2l import torch as d2l

`LSTM` 在每个时间步都有两个输出，一个是隐藏状态（通常是 `h_t`），另一个是细胞状态（通常是 `c_t`）

`GRU` 在每个时间步只有一个输出，通常是隐藏状态（`h_t`）

In [2]:
class Seq2SeqEncoder(d2l.Encoder):
    """用于序列到序列学习的循环神经网络编码器"""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                dropout=0, **kwargs):
        super(Seq2SeqEncoder, self).__init__(**kwargs)
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # Recurrent layer
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers,
                            dropout=dropout)

    def forward(self, X, *args):
        # The output `X` shape: (`batch_size`, `num_steps`, `num_hiddens`)
        # 这里因为Seq2Seq的输出是Decoder的输入，并没有经过最后的输出层，所以最后一个维度是num_hiddens
        X = self.embedding(X)
        X = X.permute(1, 0, 2) # permute(dims)将tensor的维度换位。
        # RNN, LSTM, GRU的output输出格式为:(num_steps, batch_size, num_hiddens)因为在nn.上述中没有进行输出，所以最后一个维度就是隐藏层大小
        # 如果未提及状态，则默认为0
        # state的形状:(num_layers,batch_size,num_hiddens)
        output, state = self.rnn(X)
        return output, state

In [3]:
encoder = Seq2SeqEncoder(vocab_size=10, embed_size=8, num_hiddens=16, num_layers=2)
encoder.eval() #Dropout不会生效
X = torch.zeros((4, 7), dtype=torch.long) # 4是batch_size, 7是句子的长度
output, state = encoder(X)
output.shape # torch.Size([7, 4, 16])

torch.Size([7, 4, 16])

In [4]:
# 2层，batch_size=4，num_hiddens=16
state.shape # torch.Size([2, 4, 16])

torch.Size([2, 4, 16])

In [5]:
# 这里不太懂
class Seq2SeqDecoder(d2l.Decoder):
    """用于序列到序列学习的循环神经网络解码器"""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqDecoder, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size + num_hiddens, num_hiddens, num_layers,
                          dropout=dropout)
        # 输出层
        self.dense = nn.Linear(num_hiddens, vocab_size)

    def init_state(self, enc_outputs, *args):
        return enc_outputs[1] # enc_outputs[1]是encoder的state

    def forward(self, X, state):
        # 输出'X'的形状：(batch_size,num_steps,embed_size)
        X = self.embedding(X).permute(1, 0, 2)
        # 广播context，使其具有与X相同的num_steps
        # num_layers = 2，这里拿的是第二层rnn的隐藏状态
        """
        上下文操作。这里state[-1]拿到的是“最右上角的”H(这个H融合和所有的信息)如果state是【2，4，16】的，
        那state[-1]就是【1,4,16】的。repeat重复时间步次。这样，每一个时间步都可以用到最后的H信息，
        与新的输入X做concat操作（这也是为什么解码器的self.rnn是ebd_size + num_hiddens的原因）。
        如果state[-1]是【1,4,16】，时间步是7，那重复完之后就是【7,4,16】的（7个时间步，4是batch_size，16是state隐藏单元的个数）。
        """
        context = state[-1].repeat(X.shape[0], 1, 1) # context的形状：(num_steps * 1, batch_size, num_hiddens)
        # 第1，2维不需要复制，在第0维复制num_steps次
        X_and_context = torch.cat((X, context), 2) # 在第2维拼接
        output, state = self.rnn(X_and_context, state)
        output = self.dense(output).permute(1, 0, 2)
        # output的形状:(batch_size,num_steps,vocab_size)
        # state的形状:(num_layers,batch_size,num_hiddens)
        return output, state

In [6]:
decoder = Seq2SeqDecoder(vocab_size=10, embed_size=8, num_hiddens=16,
                         num_layers=2)
decoder.eval()
state = decoder.init_state(encoder(X))
output, state = decoder(X, state)
output.shape, state.shape

(torch.Size([4, 7, 10]), torch.Size([2, 4, 16]))

# 通过零值化来屏蔽不相关的项

In [7]:
def sequence_mask(X, valid_len, value=0):
    """在序列中屏蔽不相关的项"""
    maxlen = X.size(1) # 取第一维的大小
    # None的作用主要是在使用None的位置新增一个维度。
    A = torch.arange(maxlen, dtype=torch.float32, device=X.device)[None, :] # [[0, 1, 2]]
    B = valid_len[:, None] # [[1],
                           #  [2]]
    # print(A.shape, B.shape)
    mask =  A < B
    """
    [
        [T, F, F],
        [T, T, F]
    ]
    """
    # ~表示将原来mask中为F的位置设置为0
    X[~mask] = value
    return X

X = torch.tensor([[1, 2, 3], [4, 5, 6]]) # 【2，3】
sequence_mask(X, torch.tensor([1, 2]))

tensor([[1, 0, 0],
        [4, 5, 0]])

In [ ]:
class MaskedSoftmaxCELoss(nn.CrossEntropyLoss):
    """带遮蔽的softmax的交叉熵损失函数"""
    def forward(self, pred, label, valid_len):
        weights = torch.ones_like(label)
        weights = sequence_mask(weights, valid_len)